# 1) Seteos iniciales

In [0]:
%pip install --upgrade openai

Python interpreter will be restarted.
  Attempting uninstall: distro
    Found existing installation: distro 1.4.0
    Not uninstalling distro at /usr/lib/python3/dist-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-47a91bc3-7db8-4c07-9e09-59ff8ae5977f
    Can't uninstall 'distro'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:
# librerías básicas
import os
import re
import json
import time
import random
import tempfile
import pandas as pd
from datetime import datetime
from selenium import webdriver
from pyspark.sql import SparkSession


# Librerías para scraping
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Librerias cloud Azure
import openai
from openai import AzureOpenAI

In [0]:
STORAGE_ACCOUNT_KEY = "YOUR_KEY"

In [0]:
# Set the configuration for accessing the Azure Data Lake Storage
spark.conf.set(
    "fs.azure.account.key.scrapingiastorage.dfs.core.windows.net",
    STORAGE_ACCOUNT_KEY
)    

# 2) Manejo de bloqueos

In [0]:
# Luego de numerosas pruebas, se decidió setear aleatoriamente los headers para evitar bloqueos, y no utilizar proxies, porque los proxies gratuitos son inestables y han dado varios problemas. Es mejor no setearlos en este caso.

# Lista de headers aleatorios
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:102.0) Gecko/20100101 Firefox/102.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.203",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 15_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1"
]

def get_random_headers():
    """ Esta función setea los parámetros de conexión 
    con headers aleatorios 
    """
    return {
        "User-Agent": random.choice(USER_AGENTS),
        "Accept-Language": "es-ES,es;q=0.9,en;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Connection": "keep-alive",
        "Referer": "https://www.google.com"
    }

# 3) Descarga de HTMLs con Selenium

In [0]:
# Descarga de  HTMLs 
def download_htmls(urls, tiempo_espera=5):
    """ Esta función descarga los HTMLs completos 
    utilizando scraping dinámico con beatufoulsoup 
    """
    
    html_dict = {}

    for url in urls:
        headers = get_random_headers()
        user_data_dir = tempfile.mkdtemp()

        opciones = Options()
        opciones.add_argument("--headless") 
        opciones.add_argument("--disable-gpu")
        opciones.add_argument("--no-sandbox")
        opciones.add_argument(f"--user-data-dir={user_data_dir}")
        opciones.add_argument(f"user-agent={headers['User-Agent']}")

        try:
            driver = webdriver.Chrome(options=opciones)
            driver.get(url)

            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "article, h2, h3, .headline, a[href]"))
            )

            # Scroll para cargar contenido dinámico, sirve para evitar bloqueos a bots
            for _ in range(3):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)

            time.sleep(tiempo_espera)

            html_completo = driver.page_source
            dominio = url.split("//")[-1].split("/")[0].replace("www.", "")
            html_dict[dominio] = html_completo

            driver.save_screenshot(f"screenshot_{dominio}.png")
            driver.quit()

        except Exception as e:
            print(f"Error procesando {url}: {e}")
            continue

    return html_dict

In [0]:
# Lista de urls para la descarga de sus HTMLs
urls = [
   # "https://www.marketwatch.com/latest-news",              # EE.UU.
    "https://www.cnbc.com/economy/",                         # EE.UU.
   # "https://www.nasdaq.com/news-and-insights",             # EE.UU.
    "https://www.elcomercio.pe/economia/",                   # Perú
  #  "https://www.eltiempo.com/economia",                    # Colombia
   "https://www.larepublica.co/economia",                    # Colombia
  #  "https://www.eluniversal.com.mx/cartera",               # México
    "https://www.emol.com/noticias/Economia/portada.aspx",   # Chile
  #  "https://www.ghanaweb.com/GhanaHomePage/business/",     # Ghana
  #  "https://www.businessghana.com/",                       # Ghana 
  #  "https://www.fratmat.info/economie",                    # Costa de Marfil 
  #  "https://g1.globo.com/economia/",                       # Brasil 
 #   "https://www.infomoney.com.br/",                        # Brasil 
  #  "https://www.ambito.com/economia",                      # Argentina
 #   "https://www.cronista.com/finanzas-mercados",           # Argentina
    "https://www.clarin.com/economia",                       # Argentina
    "https://www.infobae.com/economia",                      # Argentina
    "https://tn.com.ar/economia/"                            # Argentina

]

# Aplicación de la descarga de HTMLs
dict_download_htmls = download_htmls(urls)

In [0]:
# Serializar dict a JSON
json_data = json.dumps(dict_download_htmls, ensure_ascii=False, indent=2)

In [0]:
dbutils.fs.put(
    "abfss://scraps@scrapingiastorage.dfs.core.windows.net/raw/htmls.json",
    json_data,
    overwrite=True
)

Wrote 4506338 bytes.
Out[44]: True

In [0]:
# Construcción del DataFrame a partir de los HTMLs descargados
spark = SparkSession.builder.getOrCreate()

data = [(k, v) for k, v in dict_download_htmls.items()]
df_news = spark.createDataFrame(data, schema=["source", "html"])

df_news_pd = df_news.toPandas()

# 4) Credenciales y autenticación a AZURE

In [0]:
# Credenciales de conexión con Azure
YOUR_ENDPOINT = "https://scrapingia.openai.azure.com"
YOUR_IA_MODEL_NAME = "o4-mini"
DEPLOYMENT = "o4-mini-challengue"

YOUR_API_KEY = "your_key"
YOUR_API_VERSION = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=YOUR_API_VERSION,
    azure_endpoint=YOUR_ENDPOINT,
    api_key=YOUR_API_KEY,
)

# 5) Llamada al modelo de IA para Scraping

In [0]:
# Chequeo de la longitud de los HTMLs descargados
print('-'*60)
for dominio, html in dict_download_htmls.items():
    print(f"Dominio: {dominio}, longitud HTML: {len(html)}")
print('-'*60)

------------------------------------------------------------
Dominio: cnbc.com, longitud HTML: 2735541
Dominio: elcomercio.pe, longitud HTML: 312589
Dominio: larepublica.co, longitud HTML: 101647
Dominio: emol.com, longitud HTML: 123174
Dominio: clarin.com, longitud HTML: 573981
Dominio: infobae.com, longitud HTML: 311182
Dominio: tn.com.ar, longitud HTML: 194828
------------------------------------------------------------


In [0]:
def reducir_htmls_por_listas(html_dict, max_listas=5):
    """Extrae las primeras listas UL relevantes (con links) de cada HTML en el diccionario."""
    htmls_reducidos = {}
    for source, html in html_dict.items():
        try:
            soup = BeautifulSoup(html, "html.parser")
            listas = soup.find_all("ul")

            listas_filtradas = []
            for ul in listas:
                if len(ul.find_all("li")) >= 3 and ul.find("a"):
                    listas_filtradas.append(str(ul))
                if len(listas_filtradas) >= max_listas:
                    break

            htmls_reducidos[source] = "\n".join(listas_filtradas)
        except Exception as e:
            print(f"Error procesando {source}: {e}")
            htmls_reducidos[source] = ""
    return htmls_reducidos

html_dict_reducido = reducir_htmls_por_listas(dict_download_htmls)

In [0]:
# Chequeo de la longitud de los HTMLs recortados
print("-" * 60)
for dominio, html in html_dict_reducido.items():
    print(f"Dominio: {dominio}, longitud HTML: {len(html)}")
print("-" * 60)

------------------------------------------------------------
Dominio: cnbc.com, longitud HTML: 4590
Dominio: elcomercio.pe, longitud HTML: 14983
Dominio: larepublica.co, longitud HTML: 9226
Dominio: emol.com, longitud HTML: 19890
Dominio: clarin.com, longitud HTML: 39772
Dominio: infobae.com, longitud HTML: 0
Dominio: tn.com.ar, longitud HTML: 8599
------------------------------------------------------------


In [0]:
def dividir_html_en_bloques(html, max_chars=9000):
    """Divide un HTML en bloques separados si excede cierto largo (usando <ul> como unidad)."""
    soup = BeautifulSoup(html, "html.parser")
    listas = soup.find_all("ul")

    bloques = []
    bloque_actual = ""

    for ul in listas:
        ul_str = str(ul)
        if len(bloque_actual) + len(ul_str) > max_chars:
            if bloque_actual:
                bloques.append(bloque_actual)
                bloque_actual = ""
        bloque_actual += ul_str + "\n"

    if bloque_actual:
        bloques.append(bloque_actual)

    return bloques

# Si la longitud del HTML es mayor a 9000 caracteres, aplico la función para divirlo en bloques
html_dict_dividido = {source: dividir_html_en_bloques(html) for source, html in html_dict_reducido.items()}

# Chequeo de la longitud de los HTMLs divididos
print('-'*60)
for dominio, bloques in html_dict_dividido.items():
    print(f"Dominio: {dominio}, cantidad de bloques: {len(bloques)}")
    for i, bloque in enumerate(bloques):
        print(f"   → Bloque {i+1} longitud: {len(bloque)}")
print('-'*60)

------------------------------------------------------------
Dominio: cnbc.com, cantidad de bloques: 1
   → Bloque 1 longitud: 4591
Dominio: elcomercio.pe, cantidad de bloques: 2
   → Bloque 1 longitud: 6042
   → Bloque 2 longitud: 8942
Dominio: larepublica.co, cantidad de bloques: 2
   → Bloque 1 longitud: 8423
   → Bloque 2 longitud: 804
Dominio: emol.com, cantidad de bloques: 3
   → Bloque 1 longitud: 3023
   → Bloque 2 longitud: 16418
   → Bloque 3 longitud: 450
Dominio: clarin.com, cantidad de bloques: 5
   → Bloque 1 longitud: 2611
   → Bloque 2 longitud: 20549
   → Bloque 3 longitud: 925
   → Bloque 4 longitud: 10991
   → Bloque 5 longitud: 4697
Dominio: infobae.com, cantidad de bloques: 0
Dominio: tn.com.ar, cantidad de bloques: 1
   → Bloque 1 longitud: 8600
------------------------------------------------------------


In [0]:
# Función para extraer titulares y URLs con Gemini
def extraer_titulares_azure(html, source, max_retries=5):
    """Extrae titulares y URLs usando Azure OpenAI, con retry en caso de error 429."""

    for intento in range(max_retries):
        try:
            prompt = (
                "You are a data analyst. Your task is to read this reduced HTML fragment "
                "(usually containing article links) and extract a list of main news headlines.\n\n"
                "Do NOT include links to sections, categories, tags, or general pages.\n"
                "Only include individual news article headlines with their corresponding URLs.\n\n"
                "Output a valid JSON list with this exact format:\n"
                "[{\"title\": \"...\", \"url\": \"...\"}]\n\n"
                "Use double quotes for all JSON keys and values.\n"
                "Make sure the title is complete and informative. If the title seems too short, "
                "try to infer or complete it using the last part of the URL (where words are separated by hyphens).\n\n"
                "Example:\n"
                "[{\"title\": \"Man arrested for fraud in government bidding\", \"url\": \"https://example.com/news1\"}]\n\n"
                f"HTML:\n{html}"
            )

            response = client.chat.completions.create(
                model=DEPLOYMENT,
                messages=[{"role": "user", "content": prompt}]
            )

            response_text = response.choices[0].message.content
            match = re.search(r'\[.*\]', response_text, re.DOTALL)
            if not match:
                raise ValueError("No se encontró JSON en la respuesta")

            data = json.loads(match.group(0))
            for item in data:
                item["source"] = source
                item["article_date"] = datetime.now().strftime('%Y-%m-%d')
            return data

        except Exception as e:
            if "429" in str(e):
                print(f"Intento {intento + 1}/{max_retries} - Error 429 para {source}, esperando 60 segundos...")
                time.sleep(60)
            else:
                print(f"Error extrayendo con Azure para {source}: {e}")
                break

    return []

In [0]:
def extraer_todos_azure_por_bloques(html_dict_dividido, max_retries=5):
    """Procesa bloques de HTML por cada dominio y extrae titulares con Azure."""
    todas_filas = []

    for source, bloques in html_dict_dividido.items():
        print(f"Procesando {source} ({len(bloques)} bloque(s))")

        for i, bloque in enumerate(bloques):
            print(f"  → Bloque {i+1}/{len(bloques)}")
            try:
                filas = extraer_titulares_azure(bloque, source=f"{source}_bloque{i+1}", max_retries=max_retries)
                todas_filas.extend(filas)
            except Exception as e:
                print(f" Error en {source} bloque {i+1}: {e}")
    
    return pd.DataFrame(todas_filas)


In [0]:
# Procesar todos los HTMLs del diccionario
df_titulares_azure = extraer_todos_azure_por_bloques(html_dict_dividido)

Procesando cnbc.com (1 bloque(s))
  → Bloque 1/1
Procesando elcomercio.pe (2 bloque(s))
  → Bloque 1/2
  → Bloque 2/2
Procesando larepublica.co (2 bloque(s))
  → Bloque 1/2
  → Bloque 2/2
Procesando emol.com (3 bloque(s))
  → Bloque 1/3
  → Bloque 2/3
  → Bloque 3/3
Procesando clarin.com (5 bloque(s))
  → Bloque 1/5
  → Bloque 2/5
  → Bloque 3/5
  → Bloque 4/5
  → Bloque 5/5
Procesando infobae.com (0 bloque(s))
Procesando tn.com.ar (1 bloque(s))
  → Bloque 1/1


In [0]:
# Eliminar teminaciones .com , .ar, .br, .mx  de la columna 'source'
df_titulares_azure['source'] = df_titulares_azure['source'].apply(lambda x: re.sub(r'\.com|\.ar|\.br|\.co|\.pe|\.cl|\.mx|', '', x))

# Eliminar terminaciones '_bloque' de la columna 'source'
df_titulares_azure['source'] = df_titulares_azure['source'].apply(lambda x: re.sub(r'_bloque\d+$', '', x))

In [0]:
# O si la URL tiene muchas palabras separadas por guiones (característico de las noticias)
df_titulares_azure['palabras_en_url'] = df_titulares_azure['url'].str.count('-')
df_titulares_azure = df_titulares_azure[df_titulares_azure['palabras_en_url'] > 2]


In [0]:
# Agrupar noticias por source (origen) y contar cantidad de noticias obtenidas
df_titulares_azure.groupby('source').agg({'url': 'count'})

,url
source,
clarin,31
elcomercio,17
emol,12
tn,10


In [0]:
# Ordenar las columnas 
df_titulares_azure = df_titulares_azure[['article_date', 'source', 'title', 'url']]

display(df_titulares_azure)

article_date,source,title,url
2025-07-30,elcomercio,Alerta de tsunami en el Pacífico tras sismo de magnitud 8 cerca de la costa de Rusia,https://elcomercio.pe/mundo/desastres/alerta-de-tsunami-en-el-pacifico-tras-sismo-de-magnitud-8-cerca-de-la-costa-de-rusia-petropavlovsk-kamtchatski-peninsula-de-kamchatka-terremoto-temblor-sismo-alaska-usgs-ultimas-noticia/
2025-07-30,elcomercio,"Alerta de tsunami en Perú por terremoto de 8.8 en Rusia: zonas, horas y altura de las olas que afectarían la costa peruana",https://elcomercio.pe/lima/sucesos/alerta-de-tsunami-en-peru-por-terremoto-de-88-en-rusia-en-vivo-estas-son-las-zonas-las-horas-y-la-altura-de-las-olas-que-afectarian-la-costa-peruana-oceano-pacifico-marina-de-guerra-igp-coen-indeci-lbposting-noticia/
2025-07-30,elcomercio,Llegada de la mandataria Dina Boluarte a la gran parada cívico-militar por Fiestas Patrias,https://elcomercio.pe/lima/dina-boluarte-asi-fue-la-llegada-de-la-mandataria-a-la-gran-parada-civico-militar-por-fiestas-patrias-ultimas-noticia/
2025-07-30,elcomercio,Luis Arce rechaza como inadmisible frases de Dina Boluarte sobre Bolivia en su mensaje a la nación,https://elcomercio.pe/politica/actualidad/luis-arce-rechaza-como-inadmisible-frases-de-dina-boluarte-sobre-bolivia-en-su-mensaje-a-la-nacion-ultimas-noticia/
2025-07-30,elcomercio,Luis Díaz a Bayern Munich: dejó Liverpool y firmó contrato hasta 2029,https://elcomercio.pe/deporte-total/luis-diaz-a-bayern-munich-dejo-liverpool-y-firmo-contrato-hasta-2029-fichajes-noticia/
2025-07-30,elcomercio,"Último temblor en Perú: reportes de IGP, epicentro, magnitud y hora de los sismos",https://elcomercio.pe/peru/ultimo-temblor-en-peru-hoy-reportes-de-igp-epicentro-magnitud-hora-y-mas-de-sismos-censis-donde-fue-el-temblor-tdpe-noticia/
2025-07-30,elcomercio,"Después de las Fiestas Patrias 2025: conoce el próximo feriado en el Perú, fecha y quienes descansan",https://elcomercio.pe/respuestas/cuando/despues-de-las-fiestas-patrias-2025-conoce-el-proximo-feriado-a-celebrase-en-el-peru-fecha-y-quienes-descansan-segun-el-calenario-oficial-el-peruano-agosto-tdped-noticia/
2025-07-30,elcomercio,Mensaje a la Nación de Dina Boluarte: en vivo y principales anuncios por 28 de julio,https://elcomercio.pe/politica/gobierno/mensaje-a-la-nacion-en-vivo-hoy-dina-boluarte-en-directo-ver-online-ultimo-discurso-por-28-de-julio-en-el-congreso-de-la-presidenta-aumento-de-sueldo-principales-anuncios-resumen-misa-y-te-deum-lbposting-noticia/
2025-07-30,elcomercio,Paro de transportistas y marcha del 28 de julio: manifestaciones contra la inseguridad y extorsiones,https://elcomercio.pe/lima/transporte/paro-de-transportistas-y-marcha-del-28-de-julio-en-vivo-sigue-las-manifestaciones-contra-la-inseguridad-y-las-extorsiones-dina-boluarte-fiestas-patrias-lbposting-noticia/
2025-07-30,elcomercio,Municipalidad de Lima aprueba subasta de la concesión del tren Lima-Chosica al sector privado,https://elcomercio.pe/lima/municipalidad-de-lima-aprueba-subasta-de-la-concesion-del-tren-limachosica-al-sector-privado-ultimas-noticia/


In [0]:
# Conversión del Pandas DataFrame a Spark DataFrame
df_titulares_azure_spark = spark.createDataFrame(df_titulares_azure)

# Almacenamiento del Spark DataFrame
df_titulares_azure_spark.write.mode("overwrite").parquet(
    "abfss://scraps@scrapingiastorage.dfs.core.windows.net/clean/"
)

/databricks/spark/python/pyspark/sql/pandas/conversion.py:656: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]
